In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 100)
import gc

In [4]:
grid_df = pd.read_pickle('../data/input/grid_2015.pkl')
#grid_df = grid_df[(grid_df.tm_m >=4)&(grid_df.tm_m<=6)]
grid_df = grid_df.sample(frac=0.01)
grid_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
35866909,FOODS_3_101_CA_1_validation,FOODS_3_101,FOODS_3,FOODS,CA_1,CA,1580,0.0,29,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.0,137,1.0,1.000000,1.0,NaN,NaN,NaN,NaN,0,0,0,27,22,5,4,4,2,0
36240360,HOBBIES_1_180_TX_1_validation,HOBBIES_1_180,HOBBIES_1,HOBBIES,TX_1,TX,1592,0.0,306,19.984375,19.984375,19.984375,0.000000,19.984375,1.000000,1.0,4,1.0,1.000000,1.0,NaN,NaN,NaN,NaN,1,0,1,8,24,6,4,2,0,0
40068339,HOUSEHOLD_2_083_TX_1_validation,HOUSEHOLD_2_083,HOUSEHOLD_2,HOUSEHOLD,TX_1,TX,1718,0.0,7,9.968750,9.968750,9.921875,0.023987,9.953125,1.000000,2.0,31,1.0,1.001953,1.0,ColumbusDay,National,NaN,NaN,0,1,1,12,42,10,4,2,0,0
40215914,FOODS_3_452_CA_3_validation,FOODS_3_452,FOODS_3,FOODS,CA_3,CA,1723,0.0,0,4.980469,5.121094,3.980469,0.196411,4.945312,0.972656,3.0,104,1.0,1.038086,1.0,NaN,NaN,NaN,NaN,0,0,0,17,42,10,4,3,5,1
35838292,HOUSEHOLD_1_345_CA_2_validation,HOUSEHOLD_1_345,HOUSEHOLD_1,HOUSEHOLD,CA_2,CA,1579,2.0,0,4.218750,4.218750,3.720703,0.182495,3.957031,1.000000,3.0,9,1.0,1.062500,1.0,NaN,NaN,NaN,NaN,0,0,0,26,22,5,4,4,1,0


In [5]:
grid_df.shape

(110299, 34)

In [6]:
grid_df.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'release', 'sell_price', 'price_max', 'price_min', 'price_std',
       'price_mean', 'price_norm', 'price_nunique', 'item_nunique',
       'price_momentum', 'price_momentum_m', 'price_momentum_y',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y',
       'tm_wm', 'tm_dw', 'tm_w_end'],
      dtype='object')

In [9]:
# grid_may_june = grid_df[(grid_df.tm_m==5)|(grid_df.tm_m==6)]
# grid_may_june.event_name_1.unique()

In [55]:
grid_df.groupby(['store_id', 'item_id'])['sell_price'].apply(lambda x: round(x, 1).mode())

store_id  item_id           
CA_1      FOODS_1_001      0     2.199219
          FOODS_1_002      0     9.500000
          FOODS_1_003      0     3.199219
          FOODS_1_004      0     2.000000
          FOODS_1_005      0     3.500000
                                  ...    
          HOUSEHOLD_2_512  0     4.000000
          HOUSEHOLD_2_513  0     2.800781
          HOUSEHOLD_2_514  0    19.593750
          HOUSEHOLD_2_515  0     2.000000
          HOUSEHOLD_2_516  0     5.898438
Name: sell_price, Length: 3047, dtype: float16

In [11]:
cat_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 
            'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI' ]
num_cols = ['d', 'sales', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 
            'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y']
unuse_cols = ['id', 'item_id', 'cat_id', 'store_id', 'event_name_2', 'event_type_2']

In [12]:
import category_encoders as ce
list_cols = list(set(cat_cols) - set(unuse_cols))
ce_ohe = ce.OneHotEncoder(cols=list_cols,handle_unknown='impute')

In [13]:
%%time
use_cols = [col for col in grid_df.columns if col not in unuse_cols]
grid_df_one_hot = ce_ohe.fit_transform(grid_df)
grid_df_one_hot.head()

CPU times: user 759 ms, sys: 464 ms, total: 1.22 s
Wall time: 1.23 s


,id,item_id,dept_id_1,dept_id_2,dept_id_3,dept_id_4,dept_id_5,dept_id_6,dept_id_7,cat_id,store_id,state_id_1,state_id_2,state_id_3,d,sales,release,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1_1,event_name_1_2,event_name_1_3,event_name_1_4,event_name_1_5,event_name_1_6,event_name_1_7,event_name_1_8,event_name_1_9,event_name_1_10,event_name_1_11,event_name_1_12,event_name_1_13,event_name_1_14,event_name_1_15,event_name_1_16,event_name_1_17,event_name_1_18,event_name_1_19,event_name_1_20,event_name_1_21,event_name_1_22,event_name_1_23,event_name_1_24,event_name_1_25,event_name_1_26,event_name_1_27,event_name_1_28,event_name_1_29,event_name_1_30,event_name_1_31,event_type_1_1,event_type_1_2,event_type_1_3,event_type_1_4,event_type_1_5,event_name_2,event_type_2,snap_CA_1,snap_CA_2,snap_TX_1,snap_TX_2,snap_WI_1,snap_WI_2,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
35866909,FOODS_3_101_CA_1_validation,FOODS_3_101,1,0,0,0,0,0,0,FOODS,CA_1,1,0,0,1580,0.0,29,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.0,137,1.0,1.000000,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,1,0,1,0,1,0,27,22,5,4,4,2,0
36240360,HOBBIES_1_180_TX_1_validation,HOBBIES_1_180,0,1,0,0,0,0,0,HOBBIES,TX_1,0,1,0,1592,0.0,306,19.984375,19.984375,19.984375,0.000000,19.984375,1.000000,1.0,4,1.0,1.000000,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,0,1,1,0,0,1,8,24,6,4,2,0,0
40068339,HOUSEHOLD_2_083_TX_1_validation,HOUSEHOLD_2_083,0,0,1,0,0,0,0,HOUSEHOLD,TX_1,0,1,0,1718,0.0,7,9.968750,9.968750,9.921875,0.023987,9.953125,1.000000,2.0,31,1.0,1.001953,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,NaN,1,0,0,1,0,1,12,42,10,4,2,0,0
40215914,FOODS_3_452_CA_3_validation,FOODS_3_452,1,0,0,0,0,0,0,FOODS,CA_3,1,0,0,1723,0.0,0,4.980469,5.121094,3.980469,0.196411,4.945312,0.972656,3.0,104,1.0,1.038086,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,1,0,1,0,1,0,17,42,10,4,3,5,1
35838292,HOUSEHOLD_1_345_CA_2_validation,HOUSEHOLD_1_345,0,0,0,1,0,0,0,HOUSEHOLD,CA_2,1,0,0,1579,2.0,0,4.218750,4.218750,3.720703,0.182495,3.957031,1.000000,3.0,9,1.0,1.062500,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,1,0,1,0,1,0,26,22,5,4,4,1,0


In [16]:
use_cols = [col for col in grid_df_one_hot.columns if col not in unuse_cols]
#grid_df_one_hot[use_cols].info()
grid_for_ridge = grid_df_one_hot[use_cols]

In [36]:
grid_for_ridge[grid_for_ridge.isnull()]

,dept_id_1,dept_id_2,dept_id_3,dept_id_4,dept_id_5,dept_id_6,dept_id_7,state_id_1,state_id_2,state_id_3,d,sales,release,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1_1,event_name_1_2,event_name_1_3,event_name_1_4,event_name_1_5,event_name_1_6,event_name_1_7,event_name_1_8,event_name_1_9,event_name_1_10,event_name_1_11,event_name_1_12,event_name_1_13,event_name_1_14,event_name_1_15,event_name_1_16,event_name_1_17,event_name_1_18,event_name_1_19,event_name_1_20,event_name_1_21,event_name_1_22,event_name_1_23,event_name_1_24,event_name_1_25,event_name_1_26,event_name_1_27,event_name_1_28,event_name_1_29,event_name_1_30,event_name_1_31,event_type_1_1,event_type_1_2,event_type_1_3,event_type_1_4,event_type_1_5,snap_CA_1,snap_CA_2,snap_TX_1,snap_TX_2,snap_WI_1,snap_WI_2,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
35866909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36240360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40068339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40215914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35838292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32658314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40588318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39971334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32059106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
grid_for_ridge = grid_for_ridge.fillna(0)

In [38]:
TARGET      = 'sales'            
START_TRAIN = 0                  
END_TRAIN   = 1913 - 28               
P_HORIZON   = 28        

In [39]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = grid_for_ridge.drop(TARGET, axis=1)
y = grid_for_ridge[TARGET]

sc = StandardScaler()
X = sc.fit_transform(X)

In [41]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1)
ridge = Ridge()
ridge.fit(X_train, y_train)

y_preds = ridge.predict(X_valid)
score = mean_squared_error(y_valid, y_preds)
print(score)

8.928964376771098


In [42]:
print(ridge.intercept_) 
print(ridge.coef_) 

1.2466246755259758
[ 3.33449266e-01 -7.91063067e-02 -2.77946642e-01  1.07350119e-01
 -6.64885541e-03 -2.44207999e-01 -7.94416159e-03  5.57268199e-02
 -4.84382013e-02 -1.10155654e-02  1.60654051e-01 -1.59229759e-01
 -1.35713375e+00 -3.60220191e-02  2.15395146e-01 -1.03828990e-01
  7.94283956e-01 -1.58744316e-01  4.99119419e-02 -2.32538037e-01
  8.68223319e-03  1.19045221e-01  4.29919914e-02  4.23232697e-03
  1.74042517e-02 -1.53033810e-02 -1.76613457e-02  1.32614651e-02
 -1.62228841e-02  1.18463054e-02  2.37153068e-02 -1.25250628e-02
 -5.13847940e-04 -9.01513127e-03  3.10969842e-02 -3.39761387e-03
 -1.39778655e-02 -1.83591714e-03 -2.08757126e-02 -2.86056020e-03
 -1.32391080e-02 -5.31307408e-02 -2.96807345e-03 -3.26323142e-03
  3.32374372e-04 -9.36213521e-04  2.65997596e-02  5.28799934e-03
 -3.48495935e-03  7.31261662e-03  9.81163328e-03  8.56644742e-03
  1.87810018e-02 -7.83016042e-03  4.23232697e-03 -4.80053128e-03
 -7.48114728e-03 -6.94071296e-03  7.57079776e-03  5.24880715e-03
 -5.24

In [48]:
len(ridge.coef_)
len(grid_for_ridge.columns)

73

In [54]:
coef_df = pd.DataFrame({'columns': grid_for_ridge.drop('sales', axis=1).columns, 'importance': ridge.coef_})
coef_df.head(20)

,columns,importance
0,dept_id_1,0.333449
1,dept_id_2,-0.079106
2,dept_id_3,-0.277947
3,dept_id_4,0.107350
4,dept_id_5,-0.006649
5,dept_id_6,-0.244208
6,dept_id_7,-0.007944
7,state_id_1,0.055727
8,state_id_2,-0.048438
9,state_id_3,-0.011016
